In [69]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from time import time
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
import time as time
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from geopy.distance import great_circle
from shapely.geometry import MultiPoint
import progressbar

color = sns.color_palette()

%matplotlib inline

pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 999

In [70]:
import reverse_geocoder as rg

coordinates = (51.5214588,-0.1729636),(9.936033, 76.259952),(37.38605,-122.08385)

results = rg.search(coordinates) # default mode = 2

print(results)

[OrderedDict([('lat', '51.51116'), ('lon', '-0.18426'), ('name', 'Bayswater'), ('admin1', 'England'), ('admin2', 'Greater London'), ('cc', 'GB')]), OrderedDict([('lat', '9.93988'), ('lon', '76.26022'), ('name', 'Cochin'), ('admin1', 'Kerala'), ('admin2', 'Ernakulam'), ('cc', 'IN')]), OrderedDict([('lat', '37.38605'), ('lon', '-122.08385'), ('name', 'Mountain View'), ('admin1', 'California'), ('admin2', 'Santa Clara County'), ('cc', 'US')])]


In [71]:
path = r"C:\Users\kmoudgalya\Desktop\Zomato\Zomato Rating Prediction\\"
train = pd.read_csv(path + "Eatery_TrainingData_currency_normalized2.csv")
test = pd.read_csv(path + "Eatery_TestData_currency_nomalized2.csv")

In [72]:
train['dataset'] = "train"
test['dataset'] = "test"
data = pd.concat([train,test], axis = 0, ignore_index=True)

In [73]:
# Data Dictory creation with Python and replacing the values 
UserRating = pd.read_csv(path + "UserRatingTrans.csv",encoding='ISO-8859-1')
UserRating_dict = {row[0]: row[1] for row in UserRating.values}
data.user_rating__rating_text=data.user_rating__rating_text.map(UserRating_dict)
data.user_rating__rating_text.value_counts()

Good         9547
Very Good    7024
Average      6524
Not rated    5149
Excellent    1515
Poor          197
Name: user_rating__rating_text, dtype: int64

In [74]:
# URL and #N/A converted to 1 and 0 respectivily. 
data['book_url'] = data['book_url'].where(data['book_url'].isnull(), 1).fillna(0).astype(int)
data['featured_image'] = data['featured_image'].where(data['featured_image'].isnull(), 1).fillna(0).astype(int)
data['zomato_events'] = data['zomato_events'].where(data['zomato_events'].isnull(), 1).fillna(0).astype(int)
data['user_rating__custom_rating_text'] = data['user_rating__custom_rating_text'].where(data['user_rating__custom_rating_text'].isnull(), 1).fillna(0).astype(int)
data['is_book_form_web_view']=data['is_book_form_web_view'].fillna(1)
data['medio_provider']=data['medio_provider'].fillna(0)
data['is_zomato_book_res']=data['is_zomato_book_res'].fillna(0)
data['mezzo_provider']=data['mezzo_provider'].fillna('NA')

# Dealing with the names column:

In [75]:
temp_names = data['name'].value_counts()

In [76]:
temp_names = temp_names.reset_index(0)
temp_names.columns = ["restaurants","restaurant counts"]

In [77]:
temp_names

,restaurants,restaurant counts
0,McDonald's,122
1,KFC,103
2,Subway,88
3,Domino's Pizza,84
4,Baskin Robbins,71
5,Starbucks,71
6,Greggs,69
7,Pizza Hut,68
8,Burger King,66
9,Mr. Sub,60


In [78]:
temp_names2 = temp_names.where(temp_names["restaurant counts"] > 1)

In [79]:
temp_names2.dropna(inplace=True)
temp_names3 = list(temp_names2["restaurants"])
temp_names3

["McDonald's",
 'KFC',
 'Subway',
 "Domino's Pizza",
 'Baskin Robbins',
 'Starbucks',
 'Greggs',
 'Pizza Hut',
 'Burger King',
 'Mr. Sub',
 'Gong Cha',
 'Cafe Coffee Day',
 'Chatime',
 'Paul',
 "Dunkin' Donuts",
 'Krispy Kreme',
 'Cold Stone Creamery',
 'Monginis',
 "McDonald's Sorvetes",
 'Mugg & Bean',
 "Hardee's",
 'Cinnabon',
 'Perera And Sons',
 'Le Pain Quotidien',
 "Nando's",
 'Dangee Dums',
 'Starbucks Coffee',
 'Castaño',
 'Wooden Bakery',
 'Costa Coffee',
 'Tealive',
 'Shihlin',
 'Pinkberry',
 'Pappa Roti',
 "Swensen's",
 'Tim Hortons',
 'Haagen Dazs',
 'Patisserie Valerie',
 'Mado',
 "Auntie Anne's",
 'Be At One',
 'Milky Lane',
 'Boost',
 'Shamiana',
 'Mochi Mochi',
 'Mister Baker',
 'tashas',
 'Caribou Coffee',
 "Conti's",
 'Symposium Cafe Restaurant & Lounge',
 'Shakespeare and Co.',
 'KOI Cafe',
 'CoCo Fresh Tea & Juice',
 'Woolworths Cafe',
 'Cacau Show',
 'Barista',
 'The Chocolate Room',
 'Hard Rock Cafe',
 'Serenitea',
 'Lassi Shop',
 "Pain d'Or",
 'Bikanervala',
 'B

In [80]:
df_name = data[["name"]]

In [81]:
len(df_name)

30033

In [82]:
franchise_feature = []
pbar = progressbar.ProgressBar()
for row_index, row in pbar(df_name.iterrows()):
    #present_flag = 0
    if row[0]  in temp_names3:
        franchise_feature.append(1)
    else:
        franchise_feature.append(0)
    

| |                                   #           | 30032 Elapsed Time: 0:00:03


In [83]:
len(franchise_feature)

30033

In [84]:
franchise_feature

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,


In [85]:
franchise_df=pd.DataFrame({"Franchise":franchise_feature})

In [86]:
data = pd.concat([data,franchise_df], axis=1)
franchise_df

,Franchise
0,0
1,0
2,0
3,0
4,0
5,0
6,0
7,0
8,0
9,0


# Path 1:

In [87]:
# mean_cost = data['average_cost_for_two'].mean(skipna=True)
# data=data.replace({'average_cost_for_two': {0: mean_cost}}) 

In [88]:
# median_cost = data['average_cost_for_two'].median(skipna=True)
# data=data.replace({'average_cost_for_two': {0: median_cost}})

In [89]:
# mode_cost = data['average_cost_for_two'].mode()
# data=data.replace({'average_cost_for_two': {0: mode_cost}}) 

# Path 2:

In [90]:
# temp_df_list = []
# data_cost = data[['average_cost_for_two']]

In [91]:
# count1 = 0
# for row_index,row in data_cost.iterrows():
#     if row[0] == 0:
#         continue
#     else:
#         temp_df_list.append(row[0])

In [92]:
# import statistics
# path2_mean = sum(temp_df_list) / len(temp_df_list)
# path2_median = statistics.median(temp_df_list)
# path2_mode = statistics.mode(temp_df_list)

In [93]:
# data=data.replace({'average_cost_for_two': {0: path2_mean}}) 

In [94]:
# data=data.replace({'average_cost_for_two': {0: path2_median}}) 

In [95]:
# data=data.replace({'average_cost_for_two': {0: path2_mode}}) 

In [96]:
#offers, thumb and user_rating__rating_tool_tip 
#'thumb','user_rating__rating_tool_tip','offers'
data['thumb'] = data['thumb'].where(data['thumb'].isnull(),1).fillna(0).astype(int)
data['offers'] = data['offers'].where(data['offers'].isnull(),1).fillna(0).astype(int)
data['user_rating__rating_tool_tip'] = data['user_rating__rating_tool_tip'].where(data['user_rating__rating_tool_tip'].isnull(),1).fillna(0).astype(int)

In [97]:
#Column separtation
RemoveColumns = ['name','include_bogo_offers','opentable_support','switch_to_order_menu','user_rating__custom_rating_text_background']
TextColumns=['currency','establishment_types__establishment_type__id']
LocationColumns = ['location__address','location__city','location__city_id','location__country_id','location__latitude','location__locality','location__locality_verbose','location__longitude','location__zipcode']
BinaryColumns = ['has_online_delivery','has_table_booking','is_book_form_web_view','is_delivering_now','is_table_reservation_supported','is_zomato_book_res']
DummiesColumns = ['establishment_types__establishment_type__name','mezzo_provider','user_rating__rating_text']

# LATTITUDE AND LONGITUDE STUFF:

In [98]:
import math
def x_coodinator(lat, lon):
    x = math.cos(lat) * math.cos(lon)
    return x

def y_coodinator(lat, lon):
    y = math.cos(lat) * math.sin(lon)
    return y

def z_coodinator(lat):
    z = math.sin(lat) 
    return z

In [99]:
x_coord = {}
y_coord = {}
z_coord = {}

In [100]:
data_geo = data[['location__latitude', 'location__longitude', 'id']]
data_geo

,location__latitude,location__longitude,id
0,-41.305333,174.763500,18888
1,-33.912079,18.393100,14145
2,3.135924,101.686354,28502
3,38.733511,-9.143674,23643
4,14.535570,120.981812,12767
5,38.715102,-9.147563,23794
6,33.896701,35.479890,27054
7,33.891967,35.519392,26749
8,38.764475,-9.096453,23711
9,53.325217,-6.254433,30985


In [101]:
len(data_geo)

30033

In [102]:
count = 0
for row_index,row in data_geo.iterrows():
    #x_coord[row[2]] = x_coodinator(row[0], row[1])
    #y_coord[row[2]] = y_coodinator(row[0], row[1])
    #z_coord[row[2]] = z_coodinator(row[0])
    x_coord[count] = x_coodinator(row[0], row[1])
    y_coord[count] = y_coodinator(row[0], row[1])
    z_coord[count] = z_coodinator(row[0])
    count += 1
print(count)

30033


In [103]:
len(x_coord)

30033

In [104]:
x_df = pd.DataFrame.from_dict(x_coord, orient='index')
y_df = pd.DataFrame.from_dict(y_coord, orient='index')
z_df = pd.DataFrame.from_dict(z_coord, orient='index')

x_df.columns = ["X Co-ordinate"]
y_df.columns = ["Y Co-ordinate"]
z_df.columns = ["Z Co-ordinate"]

In [105]:
x_df

,X Co-ordinate
0,-0.352337
1,-0.717051
2,-0.403556
3,-0.490984
4,0.011829
5,-0.506667
6,0.476799
7,0.449910
8,-0.458323
9,-0.996240


In [106]:
#Remove columns not required
data = data[data.columns.difference(RemoveColumns)]
data = data[data.columns.difference(LocationColumns)]
data = data[data.columns.difference(TextColumns)]

In [107]:
data = pd.concat([data,x_df, y_df, z_df], axis=1)

In [108]:
cuisines_one = data.cuisines.str.get_dummies(sep = ",")
data = pd.concat([data,cuisines_one], axis=1).drop('cuisines', axis = 1)
#data = pd.concat([data,Cluster_df_col], axis=1)
data = pd.get_dummies(data, columns = DummiesColumns)

In [109]:
data.head()

,Franchise,Zomato_user_rating,average_cost_for_two,book_url,dataset,featured_image,has_online_delivery,has_table_booking,id,is_book_form_web_view,is_delivering_now,is_table_reservation_supported,is_zomato_book_res,medio_provider,offers,price_range,thumb,user_rating__custom_rating_text,user_rating__rating_tool_tip,user_rating__votes,zomato_events,X Co-ordinate,Y Co-ordinate,Z Co-ordinate,Aceh,Aegean,Afghan,Afghani,African,American,Andhra,Arabian,Argentine,Armenian,Asian,Asian Fusion,Australian,Austrian,Author,Awadhi,BBQ,Baiana,Bakery,Bakmi,Bakso,Balkans,Balti,Bangladeshi,Bar Food,Belanda,Belgian,Bengali,Betawi,Beverages,Bihari,Biryani,Black Sea,Brazilian,Breakfast,British,Bubble Tea,Burger,Burmese,Börek,Cafe,Cafe Food,Cafeteria,Caipira,Cake,Cambodian,Canadian,Cantonese,Cape Malay,Capixaba,Caribbean,Carvery,Central Anatolia,Chettinad,Chilean,Chinese,Coffee,Coffee and Tea,Colombian,Contemporary,Continental,Creole,Crepes,Cuban,Curry,Czech,Deli,Desserts,Dim Sum,Dimsum,Diner,Drinks Only,Dumplings,Durban,Döner,Eastern Anatolia,Eastern European,Egyptian,Emirati,Empanadas,Ethiopian,European,Falafel,Fast Food,Filipino,Finger Food,Fish and Chips,Fondue,Foul,French,Fresh Fish,Fried Chicken,Frozen Yogurt,Fusion,Gaúcha,Georgian,German,Giblets,Goan,Gourmet Fast Food,Greek,Grill,Gujarati,Hainanese,Hakka Chinese,Hawaiian,Healthy Food,Hokkien,Home-made,Hot Pot,Hungarian,Hyderabadi,Ice Cream,Ikan Bakar,Indian,Indonesian,International,Iranian,Irish,Italian,Izgara,Japanese,Jawa,Jewish,Juices,Kaak,Kebab,Kerala,Khaleeji,Kiwi,Korean,Korean BBQ,Kumpir,Latin American,Lebanese,Lounge,Madeiran,Maharashtrian,Malay,Malaysian,Manakish,Mandi,Mangalorean,Martabak,Mediterranean,Mexican,Middle Eastern,Mineira,Mishti,Mithai,Modern European,Momos,Mongolian,Moroccan,Mughlai,Nepalese,Nikkei,North Eastern,North Indian,Old Turkish Bars,Oriental,Others,Ottoman,Pacific,Padang,Pakistani,Pan Asian,Pastry,Patisserie,Peranakan,Peruvian,Petiscos,Pho,Pizza,Polish,Portuguese,Pub Food,Qatari,Raclette,Rajasthani,Ramen,Restaurant Cafe,Roast,Roast Chicken,Rolls,Russian,Saj,Salad,Sandwich,Satay,Scandinavian,Scottish,Seafood,Shawarma,Sichuan,Singaporean,Snack Bar,Snacks,Soto,South African,South East,South Indian,Southern,Spanish,Sri Lankan,Steak,Steamboat,Street Food,Sunda,Sushi,Swedish,Swiss,Syrian,Taiwanese,Tapas,Tea,Teochew,Teppanyaki,Tex-Mex,Thai,Tibetan,Turkish,Turkish Pizza,Vegan,Vegetarian,Venezuelan,Vietnamese,Western,World Cuisine,Wraps,Yemeni,Yum Cha,Aegean,Afghan,Afghani,African,Alentejana,American,Andhra,Arabian,Argentine,Armenian,Asian,Asian Fusion,Australian,Austrian,Author,Awadhi,BBQ,Bagels,Baiana,Bakery,Bakmi,Bakso,Balkans,Balti,Bangladeshi,Bar Food,Belgian,Bengali,Betawi,Beverages,Bihari,Biryani,Black Sea,Brasserie,Brazilian,Breakfast,British,Bubble Tea,Burger,Burmese,Börek,Cafe,Cafe Food,Cafeteria,Cajun,Canadian,Cantonese,Cape Malay,Caribbean,Carvery,Central Anatolia,Chettinad,Chifa,Chilean,Chinese,Chinese Muslim,Coffee,Coffee and Tea,Colombian,Contemporary,Continental,Crepes,Cuban,Czech,Deli,Desserts,Dim Sum,Dimsum,Diner,Drinks Only,Dumplings,Durban,Döner,Eastern Anatolia,Egyptian,Emirati,Empanadas,Ethiopian,European,Fast Food,Filipino,Finger Food,Fish and Chips,Fondue,French,Fresh Fish,Fried Chicken,Frozen Yogurt,Fusion,Georgian,German,Giblets,Goan,Gourmet Fast Food,Greek,Grill,Gujarati,Hainanese,Hakka Chinese,Hawaiian,Healthy Food,Hokkien,Home-made,Hong Kong Style,Hyderabadi,Ice Cream,Indian,Indonesian,International,Iranian,Irish,Italian,Izgara,Jamaican,Japanese,Jawa,Jewish,Juices,Kaak,Kebab,Kerala,Khaleeji,Kiwi,Korean,Kumpir,Latin American,Lebanese,Lithuanian,Lucknowi,Madeiran,Maharashtrian,Malay,Malaysian,Manado,Manakish,Mandi,Mangalorean,Mauritian,Mediterranean,Mexican,Middle Eastern,Mineira,Minhota,Mithai,Modern European,Modern Indian,Mongolian,Moroccan,Mozambican,Mughlai,Nepalese,Nikkei,North Eastern,North Indian,Northern Chinese,Old Turkish Bars,Oriental,Others,Ottoman,Paan,Pacific,Pakistani,Palembang,Pastry,Patisserie,Peranakan,Peruvian,Petiscos,Pilav,Pi

In [110]:
corr_matrix = data.corr()
corr_matrix["Zomato_user_rating"].sort_values(ascending=False)

Zomato_user_rating                                             1.000000
featured_image                                                 0.403603
thumb                                                          0.403603
user_rating__rating_text_Very Good                             0.400905
user_rating__votes                                             0.325624
user_rating__rating_text_Good                                  0.289938
user_rating__rating_text_Excellent                             0.239134
price_range                                                    0.216253
average_cost_for_two                                           0.193119
establishment_types__establishment_type__name_Casual Dining    0.157406
has_online_delivery                                            0.127290
is_table_reservation_supported                                 0.112537
is_delivering_now                                              0.108058
Y Co-ordinate                                                  0

In [111]:
# Checking Missing values # Target will get missing values 
all_data_na = (data.isnull().sum() / len(data)) * 100
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)[:30]
missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})
missing_data.head(20)

,Missing Ratio
Zomato_user_rating,20.001332


In [112]:
numerics = ['uint8','int16', 'int32', 'int64', 'float16', 'float32', 'float64']
ProcessedDataTrain = data[data.dataset == "train"] \
    .select_dtypes(include=numerics) \
    .fillna(0) 
    
ProcessedDataTest = data[data.dataset == "test"] \
    .select_dtypes(include=numerics) \
    .fillna(0) 

In [113]:
y = ProcessedDataTrain["Zomato_user_rating"]
y

0        3.7
1        3.6
2        3.8
3        3.7
4        3.4
5        4.1
6        3.7
7        4.6
8        4.0
9        3.8
10       4.7
11       0.0
12       3.0
13       4.0
14       3.8
15       3.6
16       4.3
17       3.5
18       3.6
19       0.0
20       3.5
21       3.5
22       0.0
23       3.9
24       3.2
25       3.5
26       4.5
27       4.2
28       2.7
29       0.0
        ... 
23996    4.0
23997    4.1
23998    3.7
23999    3.4
24000    4.1
24001    3.6
24002    3.9
24003    3.4
24004    3.7
24005    4.1
24006    4.4
24007    0.0
24008    3.7
24009    3.6
24010    0.0
24011    0.0
24012    0.0
24013    3.0
24014    3.6
24015    2.9
24016    3.5
24017    2.3
24018    4.2
24019    3.3
24020    0.0
24021    4.0
24022    4.7
24023    3.4
24024    4.2
24025    4.1
Name: Zomato_user_rating, Length: 24026, dtype: float64

In [114]:
#X = ProcessedDataTrain.drop(columns = ["Zomato_user_rating", "id"])
X = ProcessedDataTrain.drop(columns = ["Zomato_user_rating"])
X

,Franchise,average_cost_for_two,book_url,featured_image,has_online_delivery,has_table_booking,id,is_book_form_web_view,is_delivering_now,is_table_reservation_supported,is_zomato_book_res,medio_provider,offers,price_range,thumb,user_rating__custom_rating_text,user_rating__rating_tool_tip,user_rating__votes,zomato_events,X Co-ordinate,Y Co-ordinate,Z Co-ordinate,Aceh,Aegean,Afghan,Afghani,African,American,Andhra,Arabian,Argentine,Armenian,Asian,Asian Fusion,Australian,Austrian,Author,Awadhi,BBQ,Baiana,Bakery,Bakmi,Bakso,Balkans,Balti,Bangladeshi,Bar Food,Belanda,Belgian,Bengali,Betawi,Beverages,Bihari,Biryani,Black Sea,Brazilian,Breakfast,British,Bubble Tea,Burger,Burmese,Börek,Cafe,Cafe Food,Cafeteria,Caipira,Cake,Cambodian,Canadian,Cantonese,Cape Malay,Capixaba,Caribbean,Carvery,Central Anatolia,Chettinad,Chilean,Chinese,Coffee,Coffee and Tea,Colombian,Contemporary,Continental,Creole,Crepes,Cuban,Curry,Czech,Deli,Desserts,Dim Sum,Dimsum,Diner,Drinks Only,Dumplings,Durban,Döner,Eastern Anatolia,Eastern European,Egyptian,Emirati,Empanadas,Ethiopian,European,Falafel,Fast Food,Filipino,Finger Food,Fish and Chips,Fondue,Foul,French,Fresh Fish,Fried Chicken,Frozen Yogurt,Fusion,Gaúcha,Georgian,German,Giblets,Goan,Gourmet Fast Food,Greek,Grill,Gujarati,Hainanese,Hakka Chinese,Hawaiian,Healthy Food,Hokkien,Home-made,Hot Pot,Hungarian,Hyderabadi,Ice Cream,Ikan Bakar,Indian,Indonesian,International,Iranian,Irish,Italian,Izgara,Japanese,Jawa,Jewish,Juices,Kaak,Kebab,Kerala,Khaleeji,Kiwi,Korean,Korean BBQ,Kumpir,Latin American,Lebanese,Lounge,Madeiran,Maharashtrian,Malay,Malaysian,Manakish,Mandi,Mangalorean,Martabak,Mediterranean,Mexican,Middle Eastern,Mineira,Mishti,Mithai,Modern European,Momos,Mongolian,Moroccan,Mughlai,Nepalese,Nikkei,North Eastern,North Indian,Old Turkish Bars,Oriental,Others,Ottoman,Pacific,Padang,Pakistani,Pan Asian,Pastry,Patisserie,Peranakan,Peruvian,Petiscos,Pho,Pizza,Polish,Portuguese,Pub Food,Qatari,Raclette,Rajasthani,Ramen,Restaurant Cafe,Roast,Roast Chicken,Rolls,Russian,Saj,Salad,Sandwich,Satay,Scandinavian,Scottish,Seafood,Shawarma,Sichuan,Singaporean,Snack Bar,Snacks,Soto,South African,South East,South Indian,Southern,Spanish,Sri Lankan,Steak,Steamboat,Street Food,Sunda,Sushi,Swedish,Swiss,Syrian,Taiwanese,Tapas,Tea,Teochew,Teppanyaki,Tex-Mex,Thai,Tibetan,Turkish,Turkish Pizza,Vegan,Vegetarian,Venezuelan,Vietnamese,Western,World Cuisine,Wraps,Yemeni,Yum Cha,Aegean,Afghan,Afghani,African,Alentejana,American,Andhra,Arabian,Argentine,Armenian,Asian,Asian Fusion,Australian,Austrian,Author,Awadhi,BBQ,Bagels,Baiana,Bakery,Bakmi,Bakso,Balkans,Balti,Bangladeshi,Bar Food,Belgian,Bengali,Betawi,Beverages,Bihari,Biryani,Black Sea,Brasserie,Brazilian,Breakfast,British,Bubble Tea,Burger,Burmese,Börek,Cafe,Cafe Food,Cafeteria,Cajun,Canadian,Cantonese,Cape Malay,Caribbean,Carvery,Central Anatolia,Chettinad,Chifa,Chilean,Chinese,Chinese Muslim,Coffee,Coffee and Tea,Colombian,Contemporary,Continental,Crepes,Cuban,Czech,Deli,Desserts,Dim Sum,Dimsum,Diner,Drinks Only,Dumplings,Durban,Döner,Eastern Anatolia,Egyptian,Emirati,Empanadas,Ethiopian,European,Fast Food,Filipino,Finger Food,Fish and Chips,Fondue,French,Fresh Fish,Fried Chicken,Frozen Yogurt,Fusion,Georgian,German,Giblets,Goan,Gourmet Fast Food,Greek,Grill,Gujarati,Hainanese,Hakka Chinese,Hawaiian,Healthy Food,Hokkien,Home-made,Hong Kong Style,Hyderabadi,Ice Cream,Indian,Indonesian,International,Iranian,Irish,Italian,Izgara,Jamaican,Japanese,Jawa,Jewish,Juices,Kaak,Kebab,Kerala,Khaleeji,Kiwi,Korean,Kumpir,Latin American,Lebanese,Lithuanian,Lucknowi,Madeiran,Maharashtrian,Malay,Malaysian,Manado,Manakish,Mandi,Mangalorean,Mauritian,Mediterranean,Mexican,Middle Eastern,Mineira,Minhota,Mithai,Modern European,Modern Indian,Mongolian,Moroccan,Mozambican,Mughlai,Nepalese,Nikkei,North Eastern,North Indian,Northern Chinese,Old Turkish Bars,Oriental,Others,Ottoman,Paan,Pacific,Pakistani,Palembang,Pastry,Patisserie,Peranakan,Peruvian,Petiscos,Pilav,Pizza,Polish,Portuguese,Pub F

In [115]:
ProcessedDataTrain.to_csv('ProcessedDataTrain.csv', index = False)
ProcessedDataTest.to_csv('ProcessedDataTest.csv', index = False)

### Data Modelling 

In [116]:
#params['n_estimators'] = best_params
#print(params)

In [117]:
params = {'max_depth': 7, 'min_child_weight': 3, 'eta': 0.3, 'subsample': 1, 'colsample_bytree': 1, 'objective': 'reg:linear', 'gamma': 0.1, 'reg_alpha': 0.1, 'learning_rate': 0.05, 'n_estimators': 1000}

In [118]:
t0 = time.time()

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.1, random_state=42)

import xgboost as xgb
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

params = {'max_depth': 7, 
          'min_child_weight': 3, 
          'eta': 0.3, 
          'subsample': 1,
          'colsample_bytree': 1, 
          'objective': 'reg:linear', 
          'gamma': 0.1, 
          'reg_alpha': 0.1, 
          'learning_rate': 0.05, 
          'n_estimators': 1000}
    
num_boost_round = 999
#modelfit(xgb1, dtrain)
model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")],
    early_stopping_rounds=10
)

print("Best RMSE: {:.2f} with {} rounds".format(
                 model.best_score,
                 model.best_iteration+1)   ) 
    
# final_prediction = regr.predict(test_x)

# submission = pd.DataFrame(np.column_stack([test.id, final_prediction]), columns = ['id','Zomato_user_rating'])
# submission.to_csv("Zomato.csv", index = False)

cv_results = xgb.cv(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    seed=42,
    nfold=5,
    metrics={'rmse'},
    early_stopping_rounds=10
)

print(cv_results)

t1 = time.time()
print("time taken: " + str(t1-t0))

[18:34:07] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=5
[0]	Test-rmse:2.81979
Will train until Test-rmse hasn't improved in 10 rounds.
[18:34:08] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=5
[1]	Test-rmse:2.67928
[18:34:09] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=5
[2]	Test-rmse:2.5458
[18:34:10] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=5
[3]	Test-rmse:2.41903
[18:34:10] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=5
[4]	Test-rmse:2.29863
[18:34:11] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74:

[18:35:26] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 82 extra nodes, 24 pruned nodes, max_depth=7
[92]	Test-rmse:0.151878
[18:35:26] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 66 extra nodes, 36 pruned nodes, max_depth=7
[93]	Test-rmse:0.151609
[18:35:27] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 74 extra nodes, 32 pruned nodes, max_depth=7
[94]	Test-rmse:0.151445
[18:35:28] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 92 extra nodes, 32 pruned nodes, max_depth=7
[95]	Test-rmse:0.151209
[18:35:28] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 42 extra nodes, 50 pruned nodes, max_depth=7
[96]	Test-rmse:0.151034
[18:35:29] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 68 extra node

[18:36:37] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=5
[18:36:37] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=5
[18:36:38] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=5
[18:36:38] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=5
[18:36:39] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 2 pruned nodes, max_depth=5
[18:36:40] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=5
[18:36:41] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning e

[18:37:48] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 42 extra nodes, 2 pruned nodes, max_depth=7
[18:37:49] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 52 extra nodes, 16 pruned nodes, max_depth=7
[18:37:50] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 46 extra nodes, 10 pruned nodes, max_depth=7
[18:37:50] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 6 pruned nodes, max_depth=7
[18:37:51] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 40 extra nodes, 8 pruned nodes, max_depth=7
[18:37:52] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 56 extra nodes, 8 pruned nodes, max_depth=7
[18:37:53] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning

[18:38:55] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 74 extra nodes, 22 pruned nodes, max_depth=7
[18:38:56] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 52 extra nodes, 30 pruned nodes, max_depth=7
[18:38:56] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 22 pruned nodes, max_depth=7
[18:38:57] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 72 extra nodes, 18 pruned nodes, max_depth=7
[18:38:57] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 86 extra nodes, 22 pruned nodes, max_depth=7
[18:38:58] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 64 extra nodes, 22 pruned nodes, max_depth=7
[18:38:59] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pru

...

[18:41:07] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 56 extra nodes, 44 pruned nodes, max_depth=7
[18:41:07] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 44 extra nodes, 48 pruned nodes, max_depth=7
[18:41:08] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 64 extra nodes, 26 pruned nodes, max_depth=7
[18:41:09] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 48 extra nodes, 42 pruned nodes, max_depth=7
[18:41:09] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 74 extra nodes, 46 pruned nodes, max_depth=7
[18:41:10] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 76 extra nodes, 28 pruned nodes, max_depth=7
[18:41:10] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pru

[18:42:08] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 66 extra nodes, 58 pruned nodes, max_depth=7
[18:42:08] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 74 extra nodes, 68 pruned nodes, max_depth=7
[18:42:09] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 80 extra nodes, 72 pruned nodes, max_depth=7
[18:42:09] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 90 extra nodes, 46 pruned nodes, max_depth=7
[18:42:10] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 72 extra nodes, 64 pruned nodes, max_depth=7
[18:42:11] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 66 extra nodes, 66 pruned nodes, max_depth=7
[18:42:11] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pru

[18:43:08] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 26 extra nodes, 62 pruned nodes, max_depth=7
[18:43:09] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 34 extra nodes, 38 pruned nodes, max_depth=7
[18:43:09] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 24 extra nodes, 58 pruned nodes, max_depth=7
[18:43:10] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 24 extra nodes, 46 pruned nodes, max_depth=7
[18:43:11] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 30 pruned nodes, max_depth=7
[18:43:11] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 16 extra nodes, 62 pruned nodes, max_depth=6
[18:43:12] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pru

[18:44:13] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 72 pruned nodes, max_depth=0
[18:44:13] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 26 pruned nodes, max_depth=0
[18:44:14] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 90 pruned nodes, max_depth=0
[18:44:14] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 106 pruned nodes, max_depth=0
[18:44:15] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 52 pruned nodes, max_depth=0
[18:44:16] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 72 pruned nodes, max_depth=0
[18:44:16] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning 

[18:45:21] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 106 pruned nodes, max_depth=0
[18:45:21] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 40 pruned nodes, max_depth=0
[18:45:22] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 72 pruned nodes, max_depth=0
[18:45:23] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 28 pruned nodes, max_depth=0
[18:45:24] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 90 pruned nodes, max_depth=0
[18:45:24] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 106 pruned nodes, max_depth=0
[18:45:25] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning

In [119]:
y_label = ProcessedDataTest["Zomato_user_rating"]
df1 = ProcessedDataTest.drop(columns=["Zomato_user_rating"])

In [120]:
Data_to_predict_on = xgb.DMatrix(df1, label=y_label)

In [121]:
final_prediction = model.predict(Data_to_predict_on, ntree_limit=model.best_ntree_limit)
final_prediction.size

6007

In [122]:
submission = pd.DataFrame(np.column_stack([test.id, final_prediction]), columns = ['id','Zomato_user_rating'])
submission.to_csv("Zomato_with_latlong_chile_NAME2.csv", index = False)

# check estimators, eta and lr output first. Assign all the best values from grid search for these and NOW train a model and check RMSE. make predictions for test and make a submission

In [123]:
# from tpot import TPOTRegressor

In [124]:
# time1 = time.time()
# tpot = TPOTRegressor(generations=25, population_size=25, verbosity=2)
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.1, random_state=42)
# tpot.fit(X_train, y_train)
# time2 = time.time()
# print(tpot.score(X_test, y_test))

In [125]:
# #print("Time taken :" time2 - time1)
# print(math.sqrt(tpot.score(X_test, y_test)*-1))

In [126]:
# tpot.scoring_function

In [127]:
# final_prediction_tpot = tpot.predict(ProcessedDataTest.drop(columns = ["Zomato_user_rating"]))

In [128]:
# submission = pd.DataFrame(np.column_stack([test.id, final_prediction_tpot]), columns = ['id','Zomato_user_rating'])
# submission.to_csv("Zomato_with_TPOT2.csv", index = False)

In [129]:
# tpot.export('tpot_zomato_pipeline2.py')

In [133]:
from sklearn.model_selection import KFold
cv_groups = KFold(n_splits=10)

In [136]:
numerics = ['uint8','int16', 'int32', 'int64', 'float16', 'float32', 'float64']
train_x = data[data.dataset == "train"] \
    .select_dtypes(include=numerics) \
    .drop("Zomato_user_rating", axis = 1) \
    .fillna(0) \
    .values

test_x = data[data.dataset == "test"] \
    .select_dtypes(include=numerics) \
    .drop("Zomato_user_rating", axis = 1) \
    .fillna(0) \
    .values
    
train_y = data[data.dataset == "train"].Zomato_user_rating.values

# Ensembling with Pystack:

In [155]:
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.decomposition import PCA
from pystacknet.pystacknet import StackNetRegressor
from pystacknet.metrics import rmse,mae

model=StackNetClassifier(models, metric="auc", folds=4,
restacking=False,use_retraining=True, use_proba=True, 
random_state=12345,n_jobs=1, verbose=1)

model.fit(x,y)
preds=model.predict_proba(x_test)

models=[ 

        [           
         MLPRegressor(activation="relu",solver="lbfgs"),
         ExtraTreesRegressor(n_estimators=1000, max_depth=5, max_features=None ,random_state=0),
         MLPRegressor(activation="tanh",solver="lbfgs"),
         RandomForestRegressor(max_depth=5, random_state=0, max_features=0.5,),
         RandomForestRegressor(max_depth=10, random_state=0, max_features=0.5,),
         RandomForestRegressor(max_depth=15, random_state=0, max_features=0.5,),
         lgb.LGBMRegressor(num_leaves=31,learning_rate=0.05,n_estimators=100,max_depth = 3),
         lgb.LGBMRegressor(num_leaves=31,learning_rate=0.05,n_estimators=100,max_depth = 5),
         lgb.LGBMRegressor(num_leaves=31,learning_rate=0.05,n_estimators=100,max_depth = 8),
         Ridge(random_state=1)
         ],

        [xgb (max_depth = 7, 
              min_child_weight = 3, 
              eta = 0.3, 
              subsample = 1,
              colsample_bytree = 1, 
              objective = 'reg:linear', 
              gamma = 0.1, 
              reg_alpha = 0.1, 
              learning_rate = 0.05, 
              n_estimators = 1000)]


        ]

model=StackNetRegressor(models, metric="rmse", folds=5,restacking=False,use_retraining=True,random_state=12345,n_jobs=1, verbose=1)

model.fit(X_train,y_train)
preds=model.predict(x_test)
print ("rmse result , %f " % (rmse(y_test,preds)))

ImportError: cannot import name 'StackNetRegressor'

# Extra Trees:

In [137]:
n_jobs = 4
print("Fitting ExtraTreesregressor on faces data with %d cores..." % n_jobs)
t0 = time.time()
regr = ExtraTreesRegressor(n_estimators=1000,n_jobs=n_jobs,random_state=0)

for train_index, test_index in cv_groups.split(train_x):
    
    # Train the model using the training sets
    regr.fit(train_x[train_index], train_y[train_index])
    
    # Make predictions using the testing set
    pred_rf = regr.predict(train_x[test_index])
    
    # Calculate RMSE for current cross-validation split
    rmse = str(np.sqrt(np.mean((train_y[test_index] - pred_rf)**2)))
    
    print("RMSE for current split: " + rmse)

regr.fit(train_x, train_y)
final_prediction = regr.predict(test_x)
# submission = pd.DataFrame(np.column_stack([test.id, final_prediction]), columns = ['id','Zomato_user_rating'])
# submission.to_csv("Zomato.csv", index = False)
t1 = time.time()
print("time taken: " + str(t1-t0))

Fitting ExtraTreesregressor on faces data with 4 cores...
RMSE for current split: 0.1598174904373387
RMSE for current split: 0.1602207171313592
RMSE for current split: 0.1578824090037483
RMSE for current split: 0.16076796113760095
RMSE for current split: 0.15866870223938162


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\kmoudgalya\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2910, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-137-791508360885>", line 11, in <module>
    regr.fit(train_x[train_index], train_y[train_index])
  File "C:\Users\kmoudgalya\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py", line 328, in fit
    for i, t in enumerate(trees))
  File "C:\Users\kmoudgalya\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\externals\joblib\parallel.py", line 789, in __call__
    self.retrieve()
  File "C:\Users\kmoudgalya\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\externals\joblib\parallel.py", line 699, in retrieve
    self._output.extend(job.get(timeout=self.timeout))
  File "C:\Users\kmoudgalya\AppData\Local\Continuum\anaconda3\lib\multiprocessing\pool.py", line 638, in get
    self.wait(timeout)
  F

KeyboardInterrupt: 

# MLP Regressor 1:

In [ ]:
t0 = time.time()
regr = MLPRegressor(activation="relu",solver="lbfgs")

for train_index, test_index in cv_groups.split(train_x):
    
    # Train the model using the training sets
    regr.fit(train_x[train_index], train_y[train_index])
    
    # Make predictions using the testing set
    pred_rf = regr.predict(train_x[test_index])
    
    # Calculate RMSE for current cross-validation split
    rmse = str(np.sqrt(np.mean((train_y[test_index] - pred_rf)**2)))
    
    print("RMSE for current split: " + rmse)

regr.fit(train_x, train_y)
final_prediction = regr.predict(test_x)
# submission = pd.DataFrame(np.column_stack([test.id, final_prediction]), columns = ['id','Zomato_user_rating'])
# submission.to_csv("Zomato.csv", index = False)
t1 = time.time()
print("time taken: " + str(t1-t0))

# MLP Regressor 2:

In [ ]:
t0 = time.time()
regr = MLPRegressor(activation="tanh",solver="lbfgs")

for train_index, test_index in cv_groups.split(train_x):
    
    # Train the model using the training sets
    regr.fit(train_x[train_index], train_y[train_index])
    
    # Make predictions using the testing set
    pred_rf = regr.predict(train_x[test_index])
    
    # Calculate RMSE for current cross-validation split
    rmse = str(np.sqrt(np.mean((train_y[test_index] - pred_rf)**2)))
    
    print("RMSE for current split: " + rmse)

regr.fit(train_x, train_y)
final_prediction = regr.predict(test_x)
# submission = pd.DataFrame(np.column_stack([test.id, final_prediction]), columns = ['id','Zomato_user_rating'])
# submission.to_csv("Zomato.csv", index = False)
t1 = time.time()
print("time taken: " + str(t1-t0))

# MLP Regressor 3:

In [ ]:
t0 = time.time()
regr = MLPRegressor(activation="logistic",solver="lbfgs")

for train_index, test_index in cv_groups.split(train_x):
    
    # Train the model using the training sets
    regr.fit(train_x[train_index], train_y[train_index])
    
    # Make predictions using the testing set
    pred_rf = regr.predict(train_x[test_index])
    
    # Calculate RMSE for current cross-validation split
    rmse = str(np.sqrt(np.mean((train_y[test_index] - pred_rf)**2)))
    
    print("RMSE for current split: " + rmse)

regr.fit(train_x, train_y)
final_prediction = regr.predict(test_x)
# submission = pd.DataFrame(np.column_stack([test.id, final_prediction]), columns = ['id','Zomato_user_rating'])
# submission.to_csv("Zomato.csv", index = False)
t1 = time.time()
print("time taken: " + str(t1-t0))

# DT Regressor 1:

In [ ]:
t0 = time.time()
regr = DecisionTreeRegressor(max_depth=5)
for train_index, test_index in cv_groups.split(train_x):
    
    # Train the model using the training sets
    regr.fit(train_x[train_index], train_y[train_index])
    
    # Make predictions using the testing set
    pred_rf = regr.predict(train_x[test_index])
    
    # Calculate RMSE for current cross-validation split
    rmse = str(np.sqrt(np.mean((train_y[test_index] - pred_rf)**2)))
    
    print("RMSE for current split: " + rmse)

regr.fit(train_x, train_y)
final_prediction = regr.predict(test_x)
# submission = pd.DataFrame(np.column_stack([test.id, final_prediction]), columns = ['id','Zomato_user_rating'])
# submission.to_csv("Zomato.csv", index = False)
t1 = time.time()
print("time taken: " + str(t1-t0))

# DT Regressor 2:

In [ ]:
t0 = time.time()
regr = DecisionTreeRegressor(max_depth=10)
for train_index, test_index in cv_groups.split(train_x):
    
    # Train the model using the training sets
    regr.fit(train_x[train_index], train_y[train_index])
    
    # Make predictions using the testing set
    pred_rf = regr.predict(train_x[test_index])
    
    # Calculate RMSE for current cross-validation split
    rmse = str(np.sqrt(np.mean((train_y[test_index] - pred_rf)**2)))
    
    print("RMSE for current split: " + rmse)

regr.fit(train_x, train_y)
final_prediction = regr.predict(test_x)
# submission = pd.DataFrame(np.column_stack([test.id, final_prediction]), columns = ['id','Zomato_user_rating'])
# submission.to_csv("Zomato.csv", index = False)
t1 = time.time()
print("time taken: " + str(t1-t0))

# DT Regressor 3:

In [ ]:
t0 = time.time()
regr = DecisionTreeRegressor(max_depth=15)
for train_index, test_index in cv_groups.split(train_x):
    
    # Train the model using the training sets
    regr.fit(train_x[train_index], train_y[train_index])
    
    # Make predictions using the testing set
    pred_rf = regr.predict(train_x[test_index])
    
    # Calculate RMSE for current cross-validation split
    rmse = str(np.sqrt(np.mean((train_y[test_index] - pred_rf)**2)))
    
    print("RMSE for current split: " + rmse)

regr.fit(train_x, train_y)
final_prediction = regr.predict(test_x)
# submission = pd.DataFrame(np.column_stack([test.id, final_prediction]), columns = ['id','Zomato_user_rating'])
# submission.to_csv("Zomato.csv", index = False)
t1 = time.time()
print("time taken: " + str(t1-t0))

# ADABoost Regressor 1:

In [ ]:
t0 = time.time()
regr = AdaBoostRegressor(base_estimator=dt2, algorithm="SAMME.R")
for train_index, test_index in cv_groups.split(train_x):
    
    # Train the model using the training sets
    regr.fit(train_x[train_index], train_y[train_index])
    
    # Make predictions using the testing set
    pred_rf = regr.predict(train_x[test_index])
    
    # Calculate RMSE for current cross-validation split
    rmse = str(np.sqrt(np.mean((train_y[test_index] - pred_rf)**2)))
    
    print("RMSE for current split: " + rmse)

regr.fit(train_x, train_y)
final_prediction = regr.predict(test_x)
# submission = pd.DataFrame(np.column_stack([test.id, final_prediction]), columns = ['id','Zomato_user_rating'])
# submission.to_csv("Zomato.csv", index = False)
t1 = time.time()
print("time taken: " + str(t1-t0))

# RF Regressor 1:

In [ ]:
t0 = time.time()
regr = RandomForestRegressor(max_depth=5, random_state=0)

for train_index, test_index in cv_groups.split(train_x):
    
    # Train the model using the training sets
    regr.fit(train_x[train_index], train_y[train_index])
    
    # Make predictions using the testing set
    pred_rf = regr.predict(train_x[test_index])
    
    # Calculate RMSE for current cross-validation split
    rmse = str(np.sqrt(np.mean((train_y[test_index] - pred_rf)**2)))
    
    print("RMSE for current split: " + rmse)

regr.fit(train_x, train_y)
final_prediction = regr.predict(test_x)
# submission = pd.DataFrame(np.column_stack([test.id, final_prediction]), columns = ['id','Zomato_user_rating'])
# submission.to_csv("Zomato.csv", index = False)
t1 = time.time()
print("time taken: " + str(t1-t0))

# RF Regressor 2:

In [ ]:
t0 = time.time()
regr = RandomForestRegressor(max_depth=10, random_state=0)
for train_index, test_index in cv_groups.split(train_x):
    
    # Train the model using the training sets
    regr.fit(train_x[train_index], train_y[train_index])
    
    # Make predictions using the testing set
    pred_rf = regr.predict(train_x[test_index])
    
    # Calculate RMSE for current cross-validation split
    rmse = str(np.sqrt(np.mean((train_y[test_index] - pred_rf)**2)))
    
    print("RMSE for current split: " + rmse)

regr.fit(train_x, train_y)
final_prediction = regr.predict(test_x)
# submission = pd.DataFrame(np.column_stack([test.id, final_prediction]), columns = ['id','Zomato_user_rating'])
# submission.to_csv("Zomato.csv", index = False)
t1 = time.time()
print("time taken: " + str(t1-t0))

# RF Regressor 3:

In [ ]:
t0 = time.time()
regr = RandomForestRegressor(max_depth=15, random_state=0)
for train_index, test_index in cv_groups.split(train_x):
    
    # Train the model using the training sets
    regr.fit(train_x[train_index], train_y[train_index])
    
    # Make predictions using the testing set
    pred_rf = regr.predict(train_x[test_index])
    
    # Calculate RMSE for current cross-validation split
    rmse = str(np.sqrt(np.mean((train_y[test_index] - pred_rf)**2)))
    
    print("RMSE for current split: " + rmse)

regr.fit(train_x, train_y)
final_prediction = regr.predict(test_x)
# submission = pd.DataFrame(np.column_stack([test.id, final_prediction]), columns = ['id','Zomato_user_rating'])
# submission.to_csv("Zomato.csv", index = False)
t1 = time.time()
print("time taken: " + str(t1-t0))

# gbm 1:

In [ ]:
import lightgbm as lgb
print('Starting training...')
# train
gbm = lgb.LGBMRegressor(num_leaves=31,
                        learning_rate=0.05,
                        n_estimators=100, 
                        max_depth = 3)
gbm.fit(X_train, y_train,
        eval_set=[(X_test, y_test)],
        eval_metric='l1',
        early_stopping_rounds=5)



print('Starting predicting...')
# predict
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration_)
# eval
print('The rmse of prediction is:', mean_squared_error(y_test, y_pred) ** 0.5)

# feature importances
print('Feature importances:', list(gbm.feature_importances_))

# gbm 2:

In [ ]:
import lightgbm as lgb
print('Starting training...')
# train
gbm = lgb.LGBMRegressor(num_leaves=31,
                        learning_rate=0.05,
                        n_estimators=100, 
                        max_depth = 5)
gbm.fit(X_train, y_train,
        eval_set=[(X_test, y_test)],
        eval_metric='l1',
        early_stopping_rounds=5)



print('Starting predicting...')
# predict
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration_)
# eval
print('The rmse of prediction is:', mean_squared_error(y_test, y_pred) ** 0.5)

# feature importances
print('Feature importances:', list(gbm.feature_importances_))

# gbm 3:

In [ ]:
import lightgbm as lgb
print('Starting training...')
# train
gbm = lgb.LGBMRegressor(num_leaves=31,
                        learning_rate=0.05,
                        n_estimators=100, 
                        max_depth = 8)
gbm.fit(X_train, y_train,
        eval_set=[(X_test, y_test)],
        eval_metric='l1',
        early_stopping_rounds=5)



print('Starting predicting...')
# predict
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration_)
# eval
print('The rmse of prediction is:', mean_squared_error(y_test, y_pred) ** 0.5)

# feature importances
print('Feature importances:', list(gbm.feature_importances_))